In [400]:
'''
# male_sent_obftrain_less100.tsv has total 9610 reviews 712KB
# female_sent_obfandclass_less100.tsv has total 9707 reviews 723KB
# 上面两个数据集加在一起：all_sent_trainVAE_less100.tsv
# 现在用这两个训练

with open('../sent/ori_gender_data/male_sent_obftrain_less100.tsv','r') as f:
    mo = f.readlines()

with open('../sent/ori_gender_data/female_sent_obfandclass_less100.tsv','r') as f:
    fo = f.readlines()

allo = []
allo.extend(mo)
allo.extend(fo)

import random
random.shuffle(allo)

with open('../sent/ori_gender_data/all_sent_trainVAE_less100.tsv','w') as f:
    for line in allo:
        f.write(line)
'''

"\n# male_sent_obftrain_less100.tsv has total 9610 reviews 712KB\n# female_sent_obfandclass_less100.tsv has total 9707 reviews 723KB\n# 上面两个数据集加在一起：all_sent_trainVAE_less100.tsv\n# 现在用这两个训练\n\nwith open('../sent/ori_gender_data/male_sent_obftrain_less100.tsv','r') as f:\n    mo = f.readlines()\n\nwith open('../sent/ori_gender_data/female_sent_obfandclass_less100.tsv','r') as f:\n    fo = f.readlines()\n\nallo = []\nallo.extend(mo)\nallo.extend(fo)\n\nimport random\nrandom.shuffle(allo)\n\nwith open('../sent/ori_gender_data/all_sent_trainVAE_less100.tsv','w') as f:\n    for line in allo:\n        f.write(line)\n"

In [292]:
class Net(nn.Module):

    def __init__(self,z_dim,dropout):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        #self.conv1 = nn.Conv2d(1, 6, 5)
        #self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(z_dim, 100)
        self.fc2 = nn.Linear(100, 60)
        self.fc3 = nn.Linear(60, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        #x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        #x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        #x = x.view(-1, self.num_flat_features(x))
        #x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.fc3(x)  
        return x

In [293]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from itertools import chain
from torchtext import data


class RNN_VAE(nn.Module):
    """
    1. Hu, Zhiting, et al. "Toward controlled generation of text." ICML. 2017.
    2. Bowman, Samuel R., et al. "Generating sentences from a continuous space." arXiv preprint arXiv:1511.06349 (2015).
    3. Kim, Yoon. "Convolutional neural networks for sentence classification." arXiv preprint arXiv:1408.5882 (2014).
    """

    def __init__(self, n_vocab, h_dim, z_dim, p_word_dropout=0.3, unk_idx=0, pad_idx=1, start_idx=2, eos_idx=3, max_sent_len=15, pretrained_embeddings=None, freeze_embeddings=False, gpu=False):
        super(RNN_VAE, self).__init__()

        self.UNK_IDX = unk_idx
        self.PAD_IDX = pad_idx
        self.START_IDX = start_idx
        self.EOS_IDX = eos_idx
        self.MAX_SENT_LEN = max_sent_len

        self.n_vocab = n_vocab
        self.h_dim = h_dim
        self.z_dim = z_dim
        self.p_word_dropout = p_word_dropout

        self.gpu = gpu

        """
        Word embeddings layer
        """
        if pretrained_embeddings is None:
            self.emb_dim = h_dim
            self.word_emb = nn.Embedding(n_vocab, h_dim, self.PAD_IDX)
        else:
            self.emb_dim = pretrained_embeddings.size(1)
            self.word_emb = nn.Embedding(n_vocab, self.emb_dim, self.PAD_IDX)

            # Set pretrained embeddings
            self.word_emb.weight.data.copy_(pretrained_embeddings)

            if freeze_embeddings:
                self.word_emb.weight.requires_grad = False

        """
        Encoder is GRU with FC layers connected to last hidden unit
        """
        self.encoder = nn.GRU(self.emb_dim, h_dim)
        self.q_mu = nn.Linear(h_dim, z_dim)
        self.q_logvar = nn.Linear(h_dim, z_dim)

        """
        Decoder is GRU with `z` and `c` appended at its inputs
        """
        self.decoder = nn.GRU(self.emb_dim+z_dim, z_dim, dropout=0.3)
        self.decoder_fc = nn.Linear(z_dim, n_vocab)


        """
        Grouping the model's parameters: separating encoder, decoder, and discriminator
        """
        self.encoder_params = chain(
            self.encoder.parameters(), self.q_mu.parameters(),
            self.q_logvar.parameters()
        )

        self.decoder_params = chain(
            self.decoder.parameters(), self.decoder_fc.parameters()
        )

        self.vae_params = chain(
            self.word_emb.parameters(), self.encoder_params, self.decoder_params
        )
        self.vae_params = filter(lambda p: p.requires_grad, self.vae_params)

        """
        Use GPU if set
        """
        if self.gpu:
            self.cuda()

    def forward_encoder(self, inputs):
        """
        Inputs is batch of sentences: seq_len x mbsize
        """    
        inputs = self.word_emb(inputs)
        return self.forward_encoder_embed(inputs)

    def forward_encoder_embed(self, inputs):
        """
        Inputs is embeddings of: seq_len x mbsize x emb_dim
        """
        _, h = self.encoder(inputs, None)

        # Forward to latent
        h = h.view(-1, self.h_dim)
        mu = self.q_mu(h)
        logvar = self.q_logvar(h)

        return mu, logvar

    def sample_z(self, mu, logvar):
        """
        Reparameterization trick: z = mu + std*eps; eps ~ N(0, I)
        """
        eps = Variable(torch.randn(self.z_dim))
        eps = eps.cuda() if self.gpu else eps
        z = mu + torch.exp(logvar/2) * eps
        
        return z/z.pow(2).sum().pow(0.5)

    def sample_z_prior(self, mbsize):
        """
        Sample z ~ p(z) = N(0, I)
        """
        z = Variable(torch.randn(mbsize, self.z_dim))
        z = z.cuda() if self.gpu else z
        
        return z/z.pow(2).sum().pow(0.5)

    def forward_decoder(self, inputs, z):
        """
        Inputs must be embeddings: seq_len x mbsize
        """
        dec_inputs = self.word_dropout(inputs)

        # Forward
        seq_len = dec_inputs.size(0)

        # 1 x mbsize x (z_dim+c_dim)
        init_h = z.unsqueeze(0)
        inputs_emb = self.word_emb(dec_inputs)  # seq_len x mbsize x emb_dim
        inputs_emb = torch.cat([inputs_emb, init_h.repeat(seq_len, 1, 1)], 2)

        outputs, _ = self.decoder(inputs_emb, init_h)
        seq_len, mbsize, _ = outputs.size()

        outputs = outputs.view(seq_len*mbsize, -1)
        y = self.decoder_fc(outputs)
        y = y.view(seq_len, mbsize, self.n_vocab)

        return y


    def forward(self, sentence):
        """
        Params:
        -------
        sentence: sequence of word indices.
        use_c_prior: whether to sample `c` from prior or from `discriminator`.

        Returns:
        --------
        recon_loss: reconstruction loss of VAE.
        kl_loss: KL-div loss of VAE.
        """
        self.train()

        mbsize = sentence.size(1)

        # sentence: '<start> I want to fly <eos>'
        # enc_inputs: '<start> I want to fly <eos>'
        # dec_inputs: '<start> I want to fly <eos>'
        # dec_targets: 'I want to fly <eos> <pad>'
        pad_words = Variable(torch.LongTensor([self.PAD_IDX])).repeat(1, mbsize)
        pad_words = pad_words.cuda() if self.gpu else pad_words

        enc_inputs = sentence
        dec_inputs = sentence
        dec_targets = torch.cat([sentence[1:], pad_words], dim=0)

        # Encoder: sentence -> z
        mu, logvar = self.forward_encoder(enc_inputs)
        z = self.sample_z(mu, logvar)

        # Decoder: sentence -> y
        y = self.forward_decoder(dec_inputs, z)       
        
        recon_loss = F.cross_entropy(
            y.view(-1, self.n_vocab), dec_targets.view(-1), size_average=True
        )
        kl_loss = torch.mean(0.5 * torch.sum(torch.exp(logvar) + mu**2 - 1 - logvar, 1))

        return recon_loss, kl_loss

    def generate_sentences(self, batch_size):
        """
        Generate sentences and corresponding z of (batch_size x max_sent_len)
        """
        samples = []

        for _ in range(batch_size):
            z = self.sample_z_prior(1)
            samples.append(self.sample_sentence(z, raw=True))

        return samples

    def sample_sentence(self, z, raw=False, temp=1):
        """
        Sample single sentence from p(x|z,c) according to given temperature.
        `raw = True` means this returns sentence as in dataset which is useful
        to train discriminator. `False` means that this will return list of
        `word_idx` which is useful for evaluation.
        """
        self.eval()

        word = torch.LongTensor([self.START_IDX])
        word = word.cuda() if self.gpu else word
        word = Variable(word)  # '<start>'

        z= z.view(1, 1, -1)

        h = z

        if not isinstance(h, Variable):
            h = Variable(h)

        outputs = []

        if raw:
            outputs.append(self.START_IDX)

        for i in range(self.MAX_SENT_LEN):
            emb = self.word_emb(word).view(1, 1, -1)
            emb = torch.cat([emb, z], 2)

            output, h = self.decoder(emb, h)
            y = self.decoder_fc(output).view(-1)
            y = F.softmax(y/temp, dim=0)

            idx = torch.multinomial(y,1)

            word = Variable(torch.LongTensor([int(idx)]))
            word = word.cuda() if self.gpu else word

            idx = int(idx)

            if not raw and idx == self.EOS_IDX:
                break

            outputs.append(idx)

        # Back to default state: train
        self.train()

        if raw:
            outputs = Variable(torch.LongTensor(outputs)).unsqueeze(0)
            return outputs.cuda() if self.gpu else outputs
        else:
            return outputs

    def generate_soft_embed(self, mbsize, temp=1):
        """
        Generate soft embeddings of (mbsize x emb_dim) along with target z
        and c for each row (mbsize x {z_dim, c_dim})
        """
        samples = []
        targets_z = []

        for _ in range(mbsize):
            z = self.sample_z_prior(1)

            samples.append(self.sample_soft_embed(z, temp=1))
            targets_z.append(z)

        X_gen = torch.cat(samples, dim=0)
        targets_z = torch.cat(targets_z, dim=0)

        return X_gen, targets_z

    def sample_soft_embed(self, z, temp=1):
        """
        Sample single soft embedded sentence from p(x|z,c) and temperature.
        Soft embeddings are calculated as weighted average of word_emb
        according to p(x|z,c).
        """
        self.eval()

        z = z.view(1, 1, -1)

        word = torch.LongTensor([self.START_IDX])
        word = word.cuda() if self.gpu else word
        word = Variable(word)  # '<start>'
        emb = self.word_emb(word).view(1, 1, -1)
        emb = torch.cat([emb, z], 2)

        h = z

        if not isinstance(h, Variable):
            h = Variable(h)

        outputs = [self.word_emb(word).view(1, -1)]

        for i in range(self.MAX_SENT_LEN):
            output, h = self.decoder(emb, h)
            o = self.decoder_fc(output).view(-1)

            # Sample softmax with temperature
            y = F.softmax(o / temp, dim=0)

            # Take expectation of embedding given output prob -> soft embedding
            # <y, w> = 1 x n_vocab * n_vocab x emb_dim
            emb = y.unsqueeze(0) @ self.word_emb.weight
            emb = emb.view(1, 1, -1)

            # Save resulting soft embedding
            outputs.append(emb.view(1, -1))

            # Append with z and c for the next input
            emb = torch.cat([emb, z], 2)

        # 1 x 16 x emb_dim
        outputs = torch.cat(outputs, dim=0).unsqueeze(0)

        # Back to default state: train
        self.train()

        return outputs.cuda() if self.gpu else outputs

    def word_dropout(self, inputs):
        """
        Do word dropout: with prob `p_word_dropout`, set the word to '<unk>'.
        """
        if isinstance(inputs, Variable):
            data = inputs.data.clone()
        else:
            data = inputs.clone()

        # Sample masks: elems with val 1 will be set to <unk>
        mask = torch.from_numpy(
            np.random.binomial(1, p=self.p_word_dropout, size=tuple(data.size()))
                     .astype('uint8')
        )

        if self.gpu:
            mask = mask.cuda()

        # Set to <unk>
        data[mask] = self.UNK_IDX

        return Variable(data)

In [59]:
fTEXT = data.Field(init_token='<start>', eos_token='<eos>', lower=True, tokenize='spacy')
fLABEL = data.Field(sequential=False, unk_token=None)

#fTEXT = data.Field(tokenize='spacy')
#fLABEL = data.LabelField(tensor_type=torch.FloatTensor)

print("loading dataset female_sent_obfandclass_less100.tsv...")
ftrain  = data.TabularDataset.splits(
        path='../sent/ori_gender_data/', 
        train='female_sent_obfandclass_less100.tsv',
        format='tsv',
        fields=[('Text', fTEXT),('Label', fLABEL)])[0]

fTEXT.build_vocab(ftrain, max_size=100000, vectors="fasttext.en.300d")
fLABEL.build_vocab(ftrain)

fLABEL.vocab.stoi['1']=float(1)
fLABEL.vocab.stoi['2']=float(2)
fLABEL.vocab.stoi['3']=float(3)
fLABEL.vocab.stoi['4']=float(4)
fLABEL.vocab.stoi['5']=float(5)

loading dataset female_sent_obfandclass_less100.tsv...


In [ ]:
mTEXT = data.Field(init_token='<start>', eos_token='<eos>', lower=True, tokenize='spacy')
mLABEL = data.Field(sequential=False, unk_token=None)

#mTEXT = data.Field(tokenize='spacy')
#mLABEL = data.LabelField(tensor_type=torch.FloatTensor)

print("loading dataset male_sent_obftrain_less100.tsv...")
mtrain  = data.TabularDataset.splits(
        path='../sent/ori_gender_data/', 
        train='male_sent_obftrain_less100.tsv',
        format='tsv',
        fields=[('Text', mTEXT),('Label', mLABEL)])[0]

mTEXT.build_vocab(mtrain, max_size=100000, vectors="fasttext.en.300d")
mLABEL.build_vocab(mtrain)

mLABEL.vocab.stoi['1']=float(1)
mLABEL.vocab.stoi['2']=float(2)
mLABEL.vocab.stoi['3']=float(3)
mLABEL.vocab.stoi['4']=float(4)
mLABEL.vocab.stoi['5']=float(5)

In [60]:
allTEXT = data.Field(init_token='<start>', eos_token='<eos>', lower=True, tokenize='spacy')
allLABEL = data.Field(sequential=False, unk_token=None)

#allTEXT = data.Field(tokenize='spacy')
#allLABEL = data.LabelField(tensor_type=torch.FloatTensor)

print("loading dataset all_sent_trainVAE_less100.tsv...")
alltrain  = data.TabularDataset.splits(
        path='../sent/ori_gender_data/', 
        train='all_sent_trainVAE_less100.tsv',
        format='tsv',
        fields=[('Text', allTEXT),('Label', allLABEL)])[0]

allTEXT.build_vocab(alltrain, max_size=100000, vectors="fasttext.en.300d")
allLABEL.build_vocab(alltrain)

allLABEL.vocab.stoi['1']=float(1)
allLABEL.vocab.stoi['2']=float(2)
allLABEL.vocab.stoi['3']=float(3)
allLABEL.vocab.stoi['4']=float(4)
allLABEL.vocab.stoi['5']=float(5)

loading dataset all_sent_trainVAE_less100.tsv...


In [61]:
fTEXT.vocab = allTEXT.vocab

In [282]:
mb_size = 32
h_dim = 128
lr = 1e-3
lr_decay_every = 2000000
n_iter = 20000
log_interval = 1000
z_dim = 128

# VAE model trained on all_sent_trainVAE_less100.tsv
VAEmodel = RNN_VAE(
    len(allTEXT.vocab), h_dim, z_dim, p_word_dropout=0.3,max_sent_len=40,
    pretrained_embeddings=allTEXT.vocab.vectors, freeze_embeddings=False,
    gpu=True
)

VAEmodel.load_state_dict(torch.load('models/{}.bin'.format('pre_yelp128_all_sent_trainVAE_sph_less100'))) #VAE model

/home/guojy/anaconda3/envs/pt4/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [283]:
def get_codez(inputs):
    #VAEmodel.eval()

    # sentence: '<start> I want to fly <eos>'
    # enc_inputs: '<start> I want to fly <eos>'
    # dec_inputs: '<start> I want to fly <eos>'
    # dec_targets: 'I want to fly <eos> <pad>'
    pad_words = Variable(torch.LongTensor([VAEmodel.PAD_IDX])).repeat(1, mb_size)
    pad_words = pad_words.cuda() if VAEmodel.gpu else pad_words

    enc_inputs = inputs
    dec_inputs = inputs
    dec_targets = torch.cat([inputs[1:], pad_words], dim=0)

    # Encoder: sentence -> inputs -> z
    mu, logvar = VAEmodel.forward_encoder(enc_inputs)
    z = VAEmodel.sample_z(mu, logvar)
    return z

## 不带validation的 (可以无限for循环下去)

In [335]:
ftrain_iter = data.BucketIterator(
dataset=ftrain, batch_size=mb_size,
sort_key=lambda x: data.interleave_keys(len(x.src), len(x.trg)))

## 带有validation的 (循环完一遍就会停止)

In [344]:
ftrain_split, fvalid_split = ftrain.split(split_ratio=0.8)
ftrain_split_iter = data.BucketIterator(dataset=ftrain_split, batch_size=mb_size,sort_key=lambda x: data.interleave_keys(len(x.src), len(x.trg)))
fvalid_split_iter = data.BucketIterator(dataset=fvalid_split, batch_size=mb_size,sort_key=lambda x: data.interleave_keys(len(x.src), len(x.trg)))

In [304]:
fmodel = Net(z_dim,0.5)
print("fmodel parameters: ")
print(fmodel.parameters)

#pretrained_embeddings = fTEXT.vocab.vectors

#fmodel.embedding.weight.data.copy_(pretrained_embeddings)

import torch.optim as optim
foptimizer = optim.Adam(fmodel.parameters(),lr=0.0003)
fcriterion = nn.MSELoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
fmodel = fmodel.to(device)
fcriterion = fcriterion.to(device)

import torch.nn.functional as F

def accuracy(preds,y):
    rounded_preds = torch.round(preds)
    correct = (rounded_preds==y).float()
    acc = correct.sum()/len(correct)
    return acc

fmodel parameters: 
<bound method Module.parameters of Net(
  (fc1): Linear(in_features=128, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=60, bias=True)
  (fc3): Linear(in_features=60, out_features=1, bias=True)
  (dropout): Dropout(p=0.5)
)>


In [308]:
for batch in ftrain_iterator:
    inputs = batch.Text
    if inputs.shape[1]!=32:
        print( inputs.shape)

torch.Size([24, 22])


In [388]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train() # turns on dropout and batch normalization and allow gradient update
    
    i=0
    for batch in iterator:
        i=i+1
        if i>10000:
            break
        
        inputs = batch.Text
        labels = batch.Label
    
        try:
            z = get_codez(inputs)
        except:
            print('pass training')
            continue
        
        optimizer.zero_grad() # set accumulated gradient to 0 for every start of a batch
        
        predictions = model(z).squeeze(1)
        
        labels = torch.cuda.FloatTensor([x for x in labels])
        
        labels = labels.to(device)
        
        loss = criterion(predictions, labels)
        
        acc = accuracy(predictions, labels)
        
        loss.backward() # calculate gradient
        
        optimizer.step() # update parameters
        
        if i%200==0:
            print("train batch loss: ", loss.item())
            print("train accuracy: ", acc.item())
            print(i)
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    #return epoch_loss / len(iterator), epoch_acc / len(iterator)
    return epoch_loss / 10000, epoch_acc / 10000


def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval() #turns off dropout and batch normalization
    
    with torch.no_grad():
        i=0
        for batch in iterator:
            i=i+1
            if i>len(iterator):
                break
            
            inputs = batch.Text
            labels = batch.Label

            try:
                z = get_codez(inputs)
            except:
                print('pass evaluation')
                continue
        
            predictions = model(z).squeeze(1)
        
            labels = torch.cuda.FloatTensor([x for x in labels])

            labels = labels.to(device)

            loss = criterion(predictions, labels)

            acc = accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
            if i%50 ==0:
                print("eval batch loss: ", loss.item())
                print("eval accuracy: ", acc.item())
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

#model = torch.load('fmodel')

import timeit
#start = timeit.default_timer()

In [389]:
len(ftrain_split_iter)

243

In [390]:
batch1 = next(iter(ftrain_split_iter))
text1 = batch1.Text
label1 = batch1.Label

In [391]:
label1

tensor([ 4,  4,  5,  5,  4,  5,  4,  3,  5,  5,  5,  5,  3,  2,
         4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  3,  1,  5,  5,
         2,  5,  4,  5], device='cuda:0')

In [392]:
z = get_codez(text1)
fmodel(z).squeeze(1)

tensor([ 4.2749,  4.1136,  4.0583,  4.1647,  4.0582,  4.1479,  4.1478,
         4.0926,  4.1756,  4.1099,  3.9541,  4.0663,  4.0109,  4.1480,
         4.2084,  4.0115,  4.1626,  4.1074,  4.2228,  4.1326,  4.0249,
         3.9883,  4.2099,  4.1138,  3.9990,  4.1351,  4.0731,  4.0863,
         4.2084,  4.2000,  4.0225,  4.1069], device='cuda:0')

In [393]:
text1[:,0]

tensor([    2,     6,  1532,   187,    34,    65,    55,     6,  3101,
           23,   109,     4,     3,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1], device='cuda:0')

In [394]:
' '.join([fTEXT.vocab.itos[x] for x in text1[:,11]])

'<start> as a new yorker , i have really high pizza standards . and this place is fantastic ! <eos> <pad> <pad> <pad> <pad> <pad>'

In [395]:
label1[11]

tensor(5, device='cuda:0')

In [396]:
with open('../sent/ori_gender_data/female_sent_obfandclass_less100.tsv','r') as f:
    fo = f.readlines()

In [397]:
for line in fo:
    if "if you like pizza that taste like it has a cardboard crust" in line:
        print(line)

if you like pizza that taste like it has a cardboard crust than this is the place for you	1



In [398]:
print(1)

1


In [399]:
N_EPOCHS = 20
#print("loading previous frnn3 model...")
#model = torch.load('frnn3')
try:
    for epoch in range(N_EPOCHS):
        start = timeit.default_timer()

        train_loss, train_acc = train(fmodel, ftrain_split_iter, foptimizer, fcriterion)
        valid_loss, valid_acc = evaluate(fmodel, fvalid_split_iter, fcriterion)
        #print("saving model:   frnn8")
        #torch.save(model,'frnn8')

        print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')
        #print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%')

        stop = timeit.default_timer()
        print("time duration:    ", stop - start)


except KeyboardInterrupt:
    print("interrupt")
    print('Exiting from training early')

train batch loss:  0.8057953119277954
train accuracy:  0.25
200
pass training
train batch loss:  1.047081708908081
train accuracy:  0.28125
400
pass training
train batch loss:  1.1258511543273926
train accuracy:  0.40625
600
pass training
train batch loss:  0.8462639451026917
train accuracy:  0.1875
800
pass training
train batch loss:  1.3155440092086792
train accuracy:  0.21875
1000
train batch loss:  1.4086780548095703
train accuracy:  0.1875
1200
pass training
train batch loss:  1.35652756690979
train accuracy:  0.25
1400
pass training
train batch loss:  0.8782937526702881
train accuracy:  0.34375
1600
pass training
train batch loss:  2.0113139152526855
train accuracy:  0.25
1800
pass training
train batch loss:  1.8499486446380615
train accuracy:  0.375
2000
pass training
train batch loss:  0.8470344543457031
train accuracy:  0.25
2200
pass training
train batch loss:  1.411553144454956
train accuracy:  0.375
2400
train batch loss:  1.1270596981048584
train accuracy:  0.28125
2600
pa